# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file_to_load = "cities.csv"
data = pd.read_csv(file_to_load,low_memory=False)
pprint(data)
weather_data = data[["city","lat","lon","humidity","country"]]
weather_data


Unnamed: 0            city    lat     lon  max temp  humidity  clouds  \
0             0           kapaa  22.08 -159.32     78.80      78.0    75.0   
1             1       cape town -33.93   18.42     60.01      93.0    90.0   
2             2         rikitea -23.12 -134.97     69.35      80.0    94.0   
3             3    punta arenas -53.15  -70.92     39.20      80.0    90.0   
4             4  campbell river  50.02 -125.24     66.20      72.0    40.0   
..          ...             ...    ...     ...       ...       ...     ...   
497         497             sur  -6.88  112.21     76.50      84.0    89.0   
498         498         shimoda  49.76  129.52     58.05      92.0    98.0   
499         499          aykhal  37.72  -25.43     77.00      65.0    20.0   
500         500          killam  26.73   33.94     93.20      34.0     0.0   
501         501     gondanglegi  59.06   80.86     50.74      91.0   100.0   

     wind speed country        date  
0         11.41      US  1970-

,city,lat,lon,humidity,country
0,kapaa,22.08,-159.32,78.0,US
1,cape town,-33.93,18.42,93.0,ZA
2,rikitea,-23.12,-134.97,80.0,PF
3,punta arenas,-53.15,-70.92,80.0,CL
4,campbell river,50.02,-125.24,72.0,CA
...,...,...,...,...,...
497,sur,-6.88,112.21,84.0,ID
498,shimoda,49.76,129.52,92.0,RU
499,aykhal,37.72,-25.43,65.0,PT
500,killam,26.73,33.94,34.0,EG


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

humidity = weather_data[["humidity"]].astype(float)
lat_lon = weather_data[["lat","lon"]]
fig = gmaps.figure() 
fig.add_layer(gmaps.heatmap_layer(lat_lon, weights=(humidity["humidity"]) ))

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
utopia = data[["city","lat","lon","humidity","wind speed","max temp","country"]]
utopia = utopia.loc[(utopia["max temp"] <= 80) & (utopia["max temp"] >= 75) & (utopia["humidity"] <= 80) & (utopia["wind speed"] <=30)]
utopia

,city,lat,lon,humidity,wind speed,max temp,country
0,kapaa,22.08,-159.32,78.0,11.41,78.80,US
9,caravelas,-17.71,-39.25,65.0,9.62,77.45,BR
12,meulaboh,4.14,96.13,80.0,4.61,77.07,ID
21,miranda,-6.43,39.55,77.0,15.57,77.14,TZ
28,qaanaaq,45.03,35.38,70.0,1.50,76.59,UA
35,georgetown,38.73,-27.07,69.0,8.12,79.00,PT
47,hilo,21.39,-158.15,73.0,4.70,77.00,US
48,ilam,-9.80,-139.03,78.0,21.54,78.85,PF
50,atuona,20.89,-156.47,69.0,6.93,77.00,US
58,bengkulu,7.98,49.82,73.0,26.49,79.11,SO


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
city_coord_df = utopia[["lat","lon"]]
hotel_df = []
no_hotel_counter = 0

for i,cities in (city_coord_df.iterrows()):

    try:
        latlonkey = str(cities["lat"]) + "," + str(cities["lon"])
        gurl = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=Hotel&radius=5000&location={latlonkey}&key={g_key}"

        response = requests.get(gurl)    
        hotel_data = response.json()
        hotel_df.append(hotel_data["results"][0]["name"])
        
    except IndexError:
        hotel_df.append("No hotel near by.")
        no_hotel_counter += 1

In [17]:
utopia_hotels = utopia.copy()
utopia_hotels =utopia_hotels.reset_index()
hotel_df = pd.DataFrame(hotel_df)
utopia_hotels["hotel name"] = hotel_df.astype(str)

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

KeyError: 'hotel name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
